# Notebook content 
* Loading GMD dataset
* Playing example audio
* Dataset statistics before and after discarding samples
* Extracting MIDI onset times (observations)
* Tuning Kalman Filter parameters on training dataset

### Import packages

In [1]:
import IPython.display
import os
import sys

import matplotlib.pyplot as plt
import numpy as np

sys.path.append(os.path.join("..", "src"))

from data_loader import GmdDataLoader
from switching_kalman_filter_tracker import SwitchingKalmanFilterTracker
from tracker_evaluator import TrackerEvaluator

### Define paths

In [2]:
data_root_path = os.path.join("..", "data")
dataset_root_path = os.path.join(data_root_path, "groove")

### Define constants 

In [3]:
min_duration = 30.0

### Create generator for audio files

In [4]:
groove_data_loader = GmdDataLoader(dataset_root_path)

In [5]:
train_data_generator = groove_data_loader.get_data(split=GmdDataLoader.TRAIN_SPLIT, min_duration=min_duration, 
                                                  get_midi_onsets=True)
val_data_generator = groove_data_loader.get_data(split=GmdDataLoader.VALIDATION_SPLIT, min_duration=min_duration, 
                                                  get_midi_onsets=True)
test_data_generator = groove_data_loader.get_data(split=GmdDataLoader.TEST_SPLIT, min_duration=min_duration, 
                                                  get_midi_onsets=True)

### Get dataset statistics

In [6]:
original_n = groove_data_loader.get_dataset_size()
original_train_n = groove_data_loader.get_dataset_size(split="train")
original_validation_n = groove_data_loader.get_dataset_size(split="validation")
original_test_n = groove_data_loader.get_dataset_size(split="test")
print(("Number of samples before discarding stage - Total samples: {} - Training samples: {} - Validation samples: {} - " + 
      "Test samples: {}").format(original_n, original_train_n, original_validation_n, original_test_n))
n = groove_data_loader.get_dataset_size(min_duration = min_duration)
train_n = groove_data_loader.get_dataset_size(split="train", min_duration = min_duration)
validation_n = groove_data_loader.get_dataset_size(split="validation", min_duration = min_duration)
test_n = groove_data_loader.get_dataset_size(split="test", min_duration = min_duration)
print(("Number of samples after discarding stage - Total samples: {} - Training samples: {} - Validation samples: {} - " + 
      "Test samples: {}").format(n, train_n, validation_n, test_n))

Number of samples before discarding stage - Total samples: 1150 - Training samples: 897 - Validation samples: 124 - Test samples: 129
Number of samples after discarding stage - Total samples: 346 - Training samples: 250 - Validation samples: 41 - Test samples: 55


### Extract all training onset times from MIDI file

In [7]:
X_train = []
meta_data_rows = []
start_times = []
for x, meta_data_row, start_time in train_data_generator:
    X_train.append(x)
    meta_data_rows.append(meta_data_row)
    start_times.append(start_time)
for x, meta_data_row, start_time in val_data_generator:
    X_train.append(x)
    meta_data_rows.append(meta_data_row)
    start_times.append(start_time)    
X_train = np.array(X_train)

C:\Users\matts\AppData\Local\Temp/ipykernel_7900/1993576370.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.array(X_train)


### Extract all test onset times from MIDI file

In [8]:
X_test = []
meta_data_rows_test = []
start_times_test = []
for x, meta_data_row, start_time in test_data_generator:
    X_test.append(x)
    meta_data_rows_test.append(meta_data_row)
    start_times_test.append(start_time)
X_test = np.array(X_test)    

C:\Users\matts\AppData\Local\Temp/ipykernel_7900/2084935150.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test = np.array(X_test)


In [66]:
kwargs = {
    "onset_process_noise_std": 0.01,
    "tempo_process_noise_std": 0.005,
    "measurement_noise_std": 0.1,
    "use_ideal_switch_value": False
}
tracker = SwitchingKalmanFilterTracker(**kwargs)
print(meta_data_rows_test[0]["bpm"])
for onset in X_test[0]:
    tracker.run(onset)
print(tracker.tempo_period_to_bpm(tracker.get_tempo_estimates()[-1]))

102
[29.66737085]


### Tune switching state model for when using the ideal switching variable value. 

In [9]:
tune_params = False

In [10]:
y_train = [x["bpm"] for x in meta_data_rows]
y_test = [x["bpm"] for x in meta_data_rows_test]

In [11]:
if tune_params:
    onset_process_noise_std_vals = [0.005, 0.01, 0.05, 0.1, 0.2]
    tempo_process_noise_std_vals = [0.005, 0.01, 0.05, 0.1, 0.2]
    measurement_noise_std_vals = [0.01, 0.1, 0.2]
    evaluator = TrackerEvaluator()
    best_accuracy = -1
    for onset_noise in onset_process_noise_std_vals:
        for tempo_noise in tempo_process_noise_std_vals:
            for measurement_noise in measurement_noise_std_vals:
                # Create argument dictionary
                kwargs = {
                    "onset_process_noise_std": onset_noise,
                    "tempo_process_noise_std": tempo_noise,
                    "measurement_noise_std": measurement_noise,
                    "use_ideal_switch_value": True
                }
                # Evaluate
                accuracy = evaluator.evaluate(SwitchingKalmanFilterTracker, X_train, y_train, **kwargs)
                # Report metric if better than any other previous results
                if accuracy > best_accuracy:
                    print("Best accuracy so far: {}".format(accuracy))
                    print("Arguments: {}".format(str(kwargs)))
                    best_accuracy = accuracy
                    best_params = kwargs

### Evaluate on test set

In [12]:
evaluator = TrackerEvaluator()
kwargs = {
    "onset_process_noise_std": 0.01,
    "tempo_process_noise_std": 0.005,
    "measurement_noise_std": 0.1,
    "use_ideal_switch_value": True
}

In [13]:
accuracy = evaluator.evaluate(SwitchingKalmanFilterTracker, X_test, y_test, **kwargs)

[0.1488472]
I: 0
[0.13758878]
I: 1
[0.05647098]
I: 2
[0.13692629]
I: 3
[0.09985889]
I: 4
[0.11292209]
I: 5
[0.12431466]
I: 6
[0.17034501]
I: 7
[0.06252204]
I: 8
[0.05083979]
I: 9
[0.0443077]
I: 10
[0.08744582]
I: 11
[0.0549104]
I: 12
[0.13009742]
I: 13
[0.10489557]
I: 14
[0.07715574]
I: 15
[0.57681648]
I: 16
[0.10460314]
I: 17
[0.13870137]
I: 18
[0.12422943]
I: 19
[0.11361674]
I: 20
[0.12346799]
I: 21
[0.06266237]
I: 22
[0.1115937]
I: 23
[0.2303737]
I: 24
[0.14053274]
I: 25
[0.1060977]
I: 26
[0.74672678]
I: 27
[0.11580198]
I: 28
[0.1727311]
I: 29
[0.36510445]
I: 30
[0.13899903]
I: 31
[0.13333461]
I: 32
[0.19213294]
I: 33
[0.21882039]
I: 34
[0.06637831]
I: 35
[0.07340477]
I: 36
[0.10528426]
I: 37
[0.11611997]
I: 38
[0.12459639]
I: 39
[0.09230916]
I: 40
[0.16797082]
I: 41
[0.09613585]
I: 42
[0.21509048]
I: 43
[0.15209904]
I: 44
[0.15936684]
I: 45
[0.08301616]
I: 46
[0.3787414]
I: 47
[0.1116728]
I: 48
[0.08961768]
I: 49
[0.08428492]
I: 50
[0.0610737]
I: 51
[0.05046188]
I: 52
[0.08178515]


In [14]:
print("Test accuracy for ideal switch variable computation {}".format(accuracy))

Test accuracy for ideal switch variable computation 0.01818181818181818


### Tune PF parameters

In [15]:
tune_pf = False

In [16]:
if tune_pf:
    onset_process_noise_std_vals = [0.005, 0.01, 0.05, 0.1, 0.2]
    tempo_process_noise_std_vals = [0.005, 0.01, 0.05, 0.1, 0.2]
    measurement_noise_std_vals = [0.01, 0.1, 0.2]
    evaluator = TrackerEvaluator()
    best_accuracy = -1
    for onset_noise in onset_process_noise_std_vals:
        for tempo_noise in tempo_process_noise_std_vals:
            for measurement_noise in measurement_noise_std_vals:
                # Create argument dictionary
                kwargs = {
                    "onset_process_noise_std": onset_noise,
                    "tempo_process_noise_std": tempo_noise,
                    "measurement_noise_std": measurement_noise,
                    "use_ideal_switch_value": False
                }
                # Evaluate
                accuracy = evaluator.evaluate(SwitchingKalmanFilterTracker, X_train, y_train, **kwargs)
                # Report metric if better than any other previous results
                if accuracy > best_accuracy:
                    print("Best accuracy so far: {}".format(accuracy))
                    print("Arguments: {}".format(str(kwargs)))
                    best_accuracy = accuracy
                    best_params = kwargs

In [17]:
evaluator = TrackerEvaluator()
kwargs = {
    "onset_process_noise_std": 0.01,
    "tempo_process_noise_std": 0.005,
    "measurement_noise_std": 0.1,
    "use_ideal_switch_value": False
}

In [18]:
accuracy = evaluator.evaluate(SwitchingKalmanFilterTracker, X_train, y_train, **kwargs)

[5.56606857]
I: 0
[2.18694454]
I: 1
[3.28448137]
I: 2
[3.6987302]
I: 3
[1.87648534]
I: 4
[3.63556542]
I: 5
[2.19890673]
I: 6
[2.51422166]
I: 7
[3.89864873]
I: 8
[3.91685838]
I: 9
[3.59473319]
I: 10
[4.7180102]
I: 11
[7.16194306]
I: 12
[6.66006221]
I: 13
[0.13322852]
I: 14
[3.12869468]
I: 15
[1.99290878]
I: 16
[3.58175462]
I: 17
[3.79682253]
I: 18
[0.54370515]
I: 19
[0.00254572]
I: 20
[0.13233098]
I: 21
[3.18697108]
I: 22
[0.01305413]
I: 23
[0.15333837]
I: 24
[2.95494265]
I: 25
[3.00919149]
I: 26
[3.85414973]
I: 27
[3.48712274]
I: 28


C:\Users\matts\Desktop\KTH\EL 2320 - Applied Estimation\Project\Probabilistic-tempo-tracking\notebooks\..\src\switching_kalman_filter_tracker.py:166: RuntimeWarning: invalid value encountered in true_divide
  self.particle_weights /= self.particle_weights.sum()


[0.00709466]
I: 29
[5.02306487]
I: 30
[3.72170565]
I: 31
[0.00088597]
I: 32
[0.70968871]
I: 33
[0.00445116]
I: 34
[0.07477199]
I: 35
[2.35621134]
I: 36
[0.00089543]
I: 37
[0.10366131]
I: 38
[0.74798082]
I: 39
[0.45176014]
I: 40
[0.01503201]
I: 41
[0.92073991]
I: 42
[0.00106853]
I: 43
[9.47422127]
I: 44
[0.38522596]
I: 45
[0.00129008]
I: 46
[0.00369173]
I: 47
[2.10394572]
I: 48
[4.7489485]
I: 49
[0.00557662]
I: 50
[0.81421773]
I: 51
[0.0245276]
I: 52
[3.8038137]
I: 53
[1.90543662]
I: 54
[0.01452208]
I: 55
[0.08899876]
I: 56
[4.65176359]
I: 57
[1.19925606]
I: 58
[0.43306753]
I: 59
[2.415409]
I: 60
[0.01562187]
I: 61
[0.0044823]
I: 62


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\matts\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\matts\AppData\Local\Temp/ipykernel_7900/3803012508.py", line 1, in <module>
    accuracy = evaluator.evaluate(SwitchingKalmanFilterTracker, X_train, y_train, **kwargs)
  File "C:\Users\matts\Desktop\KTH\EL 2320 - Applied Estimation\Project\Probabilistic-tempo-tracking\notebooks\..\src\tracker_evaluator.py", line 14, in evaluate
    tempo_tracker.run(onset)
  File "C:\Users\matts\Desktop\KTH\EL 2320 - Applied Estimation\Project\Probabilistic-tempo-tracking\notebooks\..\src\switching_kalman_filter_tracker.py", line 126, in run
    self.update_switch_variable(onset_time)
  File "C:\Users\matts\Desktop\KTH\EL 2320 - Applied Estimation\Project\Probabilistic-tempo-tracking\notebooks\..\src\switching_kalman_filter_tracker.py", line 167, in update_switch_variable
    self.systematic

TypeError: object of type 'NoneType' has no len()

In [ ]:
print(accuracy)

### Report accuracy metric

In [ ]:
import math

In [ ]:
x = X_train[0]
tempo_tracker = SwitchingKalmanFilterTracker(use_ideal_switch_value=True)
for onset in x:
    tempo_tracker.run(onset)
# Get last tempo estimate in BPM
last_tempo_estimate = tempo_tracker.get_tempo_estimates()[-1]
last_tempo_estimate_bpm = tempo_tracker.tempo_period_to_bpm(last_tempo_estimate)

# Round tempo estimate to closest integer
rounded_tempo = math.ceil(last_tempo_estimate_bpm) \
    if last_tempo_estimate_bpm % 1 >= 0.5 else int(last_tempo_estimate_bpm)

print(tempo_tracker.tempo_period_to_bpm(last_tempo_estimate))